# **Causal Inference for Wind Power: Disentangling Weather Effects in New Zealand's Grid**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Done")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Done


## **Creating the folder structure**

In [ ]:
import os

# Global Config
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "Data", "Final_dataset")
PLOT_DIR = os.path.join(BASE_DIR, "Results", "Plots")
TABLE_DIR = os.path.join(BASE_DIR, "Results", "Tables")

for p in [PLOT_DIR, TABLE_DIR]:
    os.makedirs(p, exist_ok=True)


FILES = {
    "North Island": "ENHANCED_DATASET_NORTH_ISLAND.csv",
    "South Island": "ENHANCED_DATASET_SOUTH_ISLAND.csv"
}

### **EDA Suite**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import spearmanr


# Consistent Color Scheme for Distributions
ISLAND_COLORS = {
    "North Island": "#FF8C00",  # Dark Orange
    "South Island": "#1E90FF"   # Dodger Blue
}

def run_eda_suite(name, filename):
    path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(path):
        print(f"Error: {filename} not found.")
        return

    print(f"\nProcessing EDA for: {name}")

    df = pd.read_csv(path)

    # Filter out curtailment (Zero floor) for cleaner stats
    clean_df = df[df['power_mw'] > 0.1].copy()

    # Get specific color for this island
    color = ISLAND_COLORS[name]

    # --- 1. FEATURE RANKING (Spearman Correlation + P-Values) ---
    target = 'power_mw'
    features = ['wind_speed_100m', 'wind_cubed', 'theoretical_energy',
                'air_density_kgm3', 'temp_c', 'pressure_hpa',
                'turbulence_proxy', 'ramp_rate']

    stats_data = []
    for feat in features:
        corr, p_val = spearmanr(clean_df[feat], clean_df[target])
        stats_data.append({
            'Feature': feat,
            'Spearman_r': corr,
            'P_Value': p_val
        })

    # Create DataFrame and Sort
    stats_df = pd.DataFrame(stats_data).sort_values(by='Spearman_r', ascending=False)

    # Save Table
    table_path = os.path.join(TABLE_DIR, f"Feature_Correlations_{name.replace(' ', '_')}.csv")
    stats_df.to_csv(table_path, index=False)
    print(f"Saved Correlation Table: {os.path.basename(table_path)}")

    # Print Interpretation
    top_feature = stats_df.iloc[0]['Feature']
    density_corr = stats_df[stats_df['Feature'] == 'air_density_kgm3']['Spearman_r'].values[0]

    print(f"Top Predictor: {top_feature} (r={stats_df.iloc[0]['Spearman_r']:.4f})")
    if top_feature == 'theoretical_energy':
        print("Observation: Theoretical energy outperforms raw wind speed, supporting physics-based feature engineering.")


    # CORRELATION HEATMAP
    plt.figure(figsize=(10, 8))

    # Calculate full matrix for heatmap
    full_corr = clean_df[features + [target]].corr(method='spearman')

    # Create Custom Red-Green Palette
    red_green_cmap = sns.diverging_palette(15, 125, s=75, l=50, center='light', as_cmap=True)

    sns.heatmap(full_corr, annot=True, cmap=red_green_cmap, center=0, fmt=".2f", vmin=-1, vmax=1)

    plt.title(f"Spearman Correlation Matrix: {name}")
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()

    plot_path = os.path.join(PLOT_DIR, f"Correlation_Matrix_{name.replace(' ', '_')}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved Plot: {os.path.basename(plot_path)}")

    # DISTRIBUTION PLOTS
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Power Distribution
    sns.histplot(clean_df['power_mw'], bins=50, kde=True, ax=axes[0], color=color, element="step")
    axes[0].set_title(f"Power Output Distribution ({name})")
    axes[0].set_xlabel("Power (MW)")
    axes[0].set_ylabel("Frequency")

    # Wind Speed Distribution
    sns.histplot(clean_df['wind_speed_100m'], bins=50, kde=True, ax=axes[1], color=color, element="step")
    axes[1].set_title(f"Wind Speed Distribution ({name})")
    axes[1].set_xlabel("Wind Speed (m/s)")

    # Air Density Distribution
    sns.histplot(clean_df['air_density_kgm3'], bins=50, kde=True, ax=axes[2], color=color, element="step")
    axes[2].set_title(f"Air Density Distribution ({name})")
    axes[2].set_xlabel("Density (kg/m³)")

    plt.tight_layout()

    dist_path = os.path.join(PLOT_DIR, f"Distributions_{name.replace(' ', '_')}.png")
    plt.savefig(dist_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved Plot: {os.path.basename(dist_path)}")


# EXECUTION

for name, f in FILES.items():
    run_eda_suite(name, f)

## **Phase-02**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx
from statsmodels.tsa.stattools import adfuller, kpss, ccf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# VIBRANT COLOR PALETTE (For Lines/Distributions)
ISLAND_COLORS = {
    "North Island": "#FF8C00",  # Dark Orange
    "South Island": "#1E90FF"   # Dodger Blue
}

# Hexbin Colormaps
HEXBIN_MAP = {
    "North Island": "Oranges",
    "South Island": "Blues"
}

print("Starting Phase 2: EDA & Causal Structure (Red-Green Correction)")
print(f"Plots directory: {PLOT_DIR}")
print(f"Tables directory: {TABLE_DIR}")


# HELPER FUNCTIONS
def check_stationarity(series, name):
    """Runs ADF and KPSS tests."""
    series = series.dropna()
    # ADF Test
    adf_result = adfuller(series)
    # KPSS Test
    kpss_result = kpss(series, regression='c', nlags="auto")

    return {
        "Variable": name,
        "ADF Statistic": round(adf_result[0], 4),
        "ADF p-value": round(adf_result[1], 4),
        "ADF Verdict": "Stationary" if adf_result[1] < 0.05 else "Non-Stationary",
        "KPSS Statistic": round(kpss_result[0], 4),
        "KPSS p-value": round(kpss_result[1], 4),
        "KPSS Verdict": "Stationary" if kpss_result[1] > 0.05 else "Non-Stationary"
    }

def plot_dag(region_name):
    """Creates and saves the Causal DAG."""
    G = nx.DiGraph()

    # Define Nodes
    nodes = [
        ('Seasonality', 'Confounder'),
        ('Synoptic_State', 'Confounder'),
        ('Wind_Speed', 'Mediator'),
        ('Air_Density', 'Treatment'),
        ('Power', 'Outcome')
    ]
    G.add_nodes_from([n[0] for n in nodes])

    # Define Edges
    edges = [
        ('Seasonality', 'Synoptic_State'),
        ('Seasonality', 'Air_Density'),
        ('Synoptic_State', 'Wind_Speed'),
        ('Synoptic_State', 'Air_Density'),
        ('Wind_Speed', 'Power'),
        ('Air_Density', 'Power')
    ]
    G.add_edges_from(edges)

    plt.figure(figsize=(10, 6))
    pos = {
        'Seasonality': (0, 2),
        'Synoptic_State': (0, 1),
        'Wind_Speed': (-1, 0),
        'Air_Density': (1, 0),
        'Power': (0, -1)
    }

    node_colors = ['lightgray', 'lightblue', 'cyan', 'lightgreen', 'gold']

    nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=4000,
            edge_color='gray', width=2, arrowsize=20, font_weight='bold')

    # Highlight Treatment Effect
    nx.draw_networkx_edges(G, pos, edgelist=[('Air_Density', 'Power')],
                           width=3, edge_color='red', arrowsize=25)

    plt.title(f"Causal DAG: {region_name}")
    plt.savefig(os.path.join(PLOT_DIR, f"DAG_{region_name.replace(' ', '_')}.png"), dpi=300, bbox_inches='tight')
    plt.close()

    return "Adjustment Set: {Synoptic_State, Seasonality, Wind_Speed}"


# MAIN EXECUTION LOOP
for region, filename in FILES.items():
    print(f"Processing {region}")
    file_path = os.path.join(DATA_DIR, filename)

    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('timestamp')

    # Filter curtailment for cleaner plots (Wind > 5 but Power < 0.1)
    clean_df = df[~((df['wind_speed_100m'] > 5) & (df['power_mw'] <= 0.1))].copy()

    # Get colors
    island_color = ISLAND_COLORS[region]
    hex_cmap = HEXBIN_MAP[region]

    # Univariate Analysis
    print(f"[{region}] Generating Univariate Plots")

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Power
    sns.histplot(clean_df['power_mw'], bins=50, kde=True, ax=axes[0], color=island_color, element="step")
    axes[0].set_title(f"Power Output Distribution ({region})")
    axes[0].set_xlabel("Power (MW)")
    axes[0].set_ylabel("Frequency")

    # Wind
    sns.histplot(clean_df['wind_speed_100m'], bins=50, kde=True, ax=axes[1], color=island_color, element="step")
    axes[1].set_title(f"Wind Speed Distribution ({region})")
    axes[1].set_xlabel("Wind Speed (m/s)")

    # Density
    sns.histplot(clean_df['air_density_kgm3'], bins=50, kde=True, ax=axes[2], color=island_color, element="step")
    axes[2].set_title(f"Air Density Distribution ({region})")
    axes[2].set_xlabel("Density (kg/m³)")

    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"Univariate_Distributions_{region.replace(' ', '_')}.png"), dpi=300)
    plt.close()

    # Seasonality (Boxplots)
    clean_df['Month'] = clean_df.index.month
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Month', y='power_mw', data=clean_df, color=island_color)
    plt.title(f"Monthly Seasonality of Power Output: {region}")
    plt.xlabel("Month")
    plt.ylabel("Power (MW)")
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(PLOT_DIR, f"Seasonality_Boxplot_{region.replace(' ', '_')}.png"), dpi=300)
    plt.close()

    # Bivariate Analysis (Correlation)
    print(f"[{region}] Generating Correlation Matrix")

    cols = ['power_mw', 'wind_speed_100m', 'air_density_kgm3', 'temp_c', 'pressure_hpa']
    corr = clean_df[cols].corr(method='spearman')

    plt.figure(figsize=(10, 8))

    # Custom Red-Green Palette
    red_green_cmap = sns.diverging_palette(15, 125, s=75, l=50, center='light', as_cmap=True)

    sns.heatmap(corr, annot=True, cmap=red_green_cmap, center=0, fmt=".2f", vmin=-1, vmax=1)
    plt.title(f"Spearman Correlation Matrix: {region}", pad=20)


    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    plt.savefig(os.path.join(PLOT_DIR, f"Correlation_Matrix_{region.replace(' ', '_')}.png"), dpi=300, bbox_inches='tight')
    plt.close()

    # Bivariate Analysis (Power Curve)
    print(f"[{region}] Generating Power Curve")

    plt.figure(figsize=(10, 6))
    plt.hexbin(clean_df['wind_speed_100m'], clean_df['power_mw'], gridsize=50, cmap=hex_cmap, mincnt=1)
    plt.colorbar(label='Count of Hours')
    plt.title(f"Empirical Power Curve: {region}")
    plt.xlabel("Wind Speed (m/s)")
    plt.ylabel("Power (MW)")
    plt.grid(True, alpha=0.2)
    plt.savefig(os.path.join(PLOT_DIR, f"Power_Curve_{region.replace(' ', '_')}.png"), dpi=300)
    plt.close()

    # Bivariate Analysis (Polar Plot - Wind Direction)
    print(f"[{region}] Generating Polar Plot")

    # Bin wind direction (10 degree bins)
    clean_df['dir_bin'] = (clean_df['wind_dir_100m'] // 10) * 10
    dir_stats = clean_df.groupby('dir_bin')['power_mw'].mean()

    # Convert to radians
    theta = np.deg2rad(dir_stats.index)
    radii = dir_stats.values

    plt.figure(figsize=(8, 8))
    ax = plt.subplot(111, projection='polar')
    ax.plot(theta, radii, color=island_color, linewidth=2)
    ax.fill(theta, radii, alpha=0.3, color=island_color)

    # Set North to top
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)


    plt.title(f"Mean Power by Wind Direction: {region}", pad=20)

    plt.savefig(os.path.join(PLOT_DIR, f"Polar_Plot_{region.replace(' ', '_')}.png"), dpi=300, bbox_inches='tight')
    plt.close()

    # Lag Analysis
    print(f"[{region}] Performing Lag Analysis")
    lags = np.arange(0, 25)
    wind = clean_df['wind_speed_100m']
    power = clean_df['power_mw']
    ccf_vals = ccf(wind, power, adjusted=False)[:25]

    plt.figure(figsize=(10, 5))
    plt.stem(lags, ccf_vals, linefmt=island_color, markerfmt=island_color)
    plt.title(f"Lag Correlation (Wind vs Power): {region}")
    plt.xlabel("Lag (Hours)")
    plt.ylabel("Correlation Coefficient")
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(PLOT_DIR, f"Lag_Analysis_{region.replace(' ', '_')}.png"), dpi=300)
    plt.close()

    # Time Series Decomposition
    print(f"[{region}] Performing Time Series Decomposition")
    daily_df = clean_df.resample('D').mean().dropna()
    res = seasonal_decompose(daily_df['power_mw'], model='additive', period=365)

    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

    res.observed.plot(ax=ax1, title='Observed', color=island_color, linewidth=1)
    res.trend.plot(ax=ax2, title='Trend (365-Day Moving Avg)', color=island_color, linewidth=2)
    res.seasonal.plot(ax=ax3, title='Seasonal Component', color=island_color, linewidth=1)
    res.resid.plot(ax=ax4, title='Residuals', color='gray', alpha=0.5, linewidth=0.5)

    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"Decomposition_{region.replace(' ', '_')}.png"), dpi=300)
    plt.close()

    # ACF/PACF
    print(f"[{region}] Generating ACF/PACF")
    residuals = res.resid.dropna()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

    plot_acf(residuals, lags=40, ax=ax1, title=f"ACF of Residuals: {region}", color=island_color, vlines_kwargs={"colors": island_color})
    plot_pacf(residuals, lags=40, ax=ax2, title=f"PACF of Residuals: {region}", color=island_color, vlines_kwargs={"colors": island_color})
    plt.savefig(os.path.join(PLOT_DIR, f"ACF_PACF_Residuals_{region.replace(' ', '_')}.png"), dpi=300)
    plt.close()

    # Stationarity Tests
    print(f"[{region}] Running Stationarity Tests...")
    stat_results = []
    stat_results.append(check_stationarity(clean_df['power_mw'], 'Power'))
    stat_results.append(check_stationarity(clean_df['wind_speed_100m'], 'Wind Speed'))

    stat_df = pd.DataFrame(stat_results)
    stat_path = os.path.join(TABLE_DIR, f"Stationarity_Tests_{region.replace(' ', '_')}.csv")
    stat_df.to_csv(stat_path, index=False)

    # DAG & Adjustment Set
    print(f"[{region}] Generating Causal DAG")
    adj_set = plot_dag(region)

    adj_path = os.path.join(TABLE_DIR, f"Adjustment_Set_{region.replace(' ', '_')}.txt")
    print("Phase 2 Complete. All artifacts saved.")

## **Phase-03**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, KFold, RandomizedSearchCV
from sklearn.base import clone
from sklearn.metrics import r2_score
from scipy import stats
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Causal Model Definition
Y_COL = 'power_mw'
T_COL = 'air_density_kgm3'

# Excluded temp/pressure to avoid multicollinearity
X_COLS = [
    'wind_speed_100m', 'wind_cubed', 'turbulence_proxy', 'ramp_rate', 'wind_lag_1',
    'wind_sin', 'wind_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos'
]

# reporting scale
EFFECT_SCALE = 0.1
EFFECT_COL = "Effect (MW per 0.1 kg/m3)"


all_results = []

# HELPER FUNCTIONS
def tune_hyperparameters(X, y):
    """
    Hyperparameter tuning using TimeSeriesSplit to prevent look-ahead bias.
    """
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9]
    }

    xgb = XGBRegressor(n_jobs=-1, random_state=42)
    tscv = TimeSeriesSplit(n_splits=3)

    search = RandomizedSearchCV(
        xgb, param_grid, n_iter=10, cv=tscv,
        scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, verbose=0
    )
    search.fit(X, y)
    return search.best_estimator_


def run_dml_manual(df, model_y, model_t, n_folds=5):
    """
    Manual DML implementation using Block K-Fold cross-fitting.
    Ensures every data point is used for testing exactly once.
    """
    kf = KFold(n_splits=n_folds, shuffle=False)
    X_data = df[X_COLS].values

    residuals_y = np.zeros(len(df))
    residuals_t = np.zeros(len(df))

    # Cross-fitting loop
    for train_idx, test_idx in kf.split(X_data):
        X_train, X_test = X_data[train_idx], X_data[test_idx]
        y_train = df[Y_COL].iloc[train_idx].values
        y_test = df[Y_COL].iloc[test_idx].values
        t_train = df[T_COL].iloc[train_idx].values
        t_test = df[T_COL].iloc[test_idx].values

        # Fit outcome model
        model_y_fold = clone(model_y)
        model_y_fold.fit(X_train, y_train)
        residuals_y[test_idx] = y_test - model_y_fold.predict(X_test)

        # Fit treatment model
        model_t_fold = clone(model_t)
        model_t_fold.fit(X_train, t_train)
        residuals_t[test_idx] = t_test - model_t_fold.predict(X_test)

    # DML: Regress outcome residuals on treatment residuals
    coef, intercept, r_value, p_value, se = stats.linregress(residuals_t, residuals_y)

    # 95% CI
    ci_lower = coef - 1.96 * se
    ci_upper = coef + 1.96 * se

    # Calculate E-value (Sensitivity to unobserved confounding)
    e_value = np.nan
    if se > 0:
        ci_lower_abs = abs(coef) - 1.96 * se
        if ci_lower_abs > 0:
            e_value = abs(coef) / abs(coef - ci_lower_abs)
        else:
            e_value = 1.0

    return {
        'Effect': coef,
        'SE': se,
        'P_Value': p_value,
        'CI_Lower': ci_lower,
        'CI_Upper': ci_upper,
        'E_Value': round(e_value, 2) if not np.isnan(e_value) else np.nan,
        'Residuals_Y': residuals_y,
        'Residuals_T': residuals_t
    }


def calculate_diagnostics(df, model_y, model_t):
    """Calculate R2 diagnostics for nuisance models."""
    model_y.fit(df[X_COLS], df[Y_COL])
    r2_y = r2_score(df[Y_COL], model_y.predict(df[X_COLS]))

    model_t.fit(df[X_COLS], df[T_COL])
    r2_t = r2_score(df[T_COL], model_t.predict(df[X_COLS]))

    return r2_y, r2_t


def process_subgroup(sub_df, region, subgroup_name, global_model_y, global_model_t):
    """Process single subgroup with adaptive retuning."""
    # Strict sample size check
    if len(sub_df) < 5000:
        return None

    # Adaptive retuning for large subgroups
    if len(sub_df) > 10000:
        tune_sample = sub_df.sample(n=min(len(sub_df), 20000), random_state=42).sort_index()
        model_y = tune_hyperparameters(tune_sample[X_COLS], tune_sample[Y_COL])
        model_t = tune_hyperparameters(tune_sample[X_COLS], tune_sample[T_COL])
    else:
        model_y = clone(global_model_y)
        model_t = clone(global_model_t)

    # R2 diagnostics
    r2_y, r2_t = calculate_diagnostics(sub_df, model_y, model_t)


    # DML fitting
    try:
        dml_result = run_dml_manual(sub_df, model_y, model_t, n_folds=3)

        # scale outputs to "per 0.1 kg/m³" ----
        eff = dml_result['Effect'] * EFFECT_SCALE
        se = dml_result['SE'] * EFFECT_SCALE
        ci_l = dml_result['CI_Lower'] * EFFECT_SCALE
        ci_u = dml_result['CI_Upper'] * EFFECT_SCALE
        ---

        return {
            'Region': region,
            'Subgroup': subgroup_name,
            EFFECT_COL: eff,
            'SE': se,
            'P-Value': dml_result['P_Value'],
            'CI_Lower': ci_l,
            'CI_Upper': ci_u,
            'N': len(sub_df),
            'R2_Outcome': r2_y,
            'R2_Treatment': r2_t,
            'E_Value': dml_result['E_Value']
        }
    except Exception as e:
        print(f"Error in {subgroup_name}: {str(e)[:50]}")
        return None


def run_dml_analysis(region_name, df):
    """Main DML analysis pipeline."""
    print(f"\n{region_name}")
    print("-" * 60)

    df = df.dropna(subset=[Y_COL, T_COL] + X_COLS)
    print(f"Dataset: {len(df):,} rows")

    # Global tuning
    print("Tuning global models (50k sample)")
    if len(df) > 50000:
        tune_df = df.sample(n=50000, random_state=42).sort_index()
    else:
        tune_df = df

    global_model_y = tune_hyperparameters(tune_df[X_COLS], tune_df[Y_COL])
    global_model_t = tune_hyperparameters(tune_df[X_COLS], tune_df[T_COL])

    # Overall effect
    print("Fitting overall effect")
    res = process_subgroup(df, region_name, "Overall", global_model_y, global_model_t)
    if res:
        all_results.append(res)
        print(f"Overall Effect: {res[EFFECT_COL]:.3f} MW per 0.1 kg/m³")

        # Residual plots
        print("Generating residual diagnostics...")
        dml_res = run_dml_manual(df, global_model_y, global_model_t, n_folds=5)

        fig, ax = plt.subplots(1, 2, figsize=(14, 5))
        color = '#FF8C00' if region_name == "North Island" else '#1E90FF'

        sns.histplot(dml_res['Residuals_Y'], bins=50, kde=True, ax=ax[0], color=color, element="step")
        ax[0].set_title(f"Outcome Model Residuals: {region_name}", fontsize=12, fontweight='bold')
        ax[0].set_xlabel("Residual Power (MW)")
        ax[0].grid(True, alpha=0.3)

        sns.histplot(dml_res['Residuals_T'], bins=50, kde=True, ax=ax[1], color='gray', element="step")
        ax[1].set_title(f"Treatment Model Residuals: {region_name}", fontsize=12, fontweight='bold')
        ax[1].set_xlabel("Residual Density (kg/m³)")
        ax[1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(
            os.path.join(PLOT_DIR, f"Residuals_Distribution_{region_name.replace(' ', '_')}.png"),
            dpi=300, bbox_inches='tight'
        )
        plt.close()

    # Subgroup analysis
    print("Running heterogeneous effects analysis...")

    season_map = {
        12: 'Summer', 1: 'Summer', 2: 'Summer',
        3: 'Autumn', 4: 'Autumn', 5: 'Autumn',
        6: 'Winter', 7: 'Winter', 8: 'Winter',
        9: 'Spring', 10: 'Spring', 11: 'Spring'
    }
    df['Season'] = df.index.month.map(season_map)
    df['Is_Day'] = (df.index.hour >= 6) & (df.index.hour < 18)

    subgroups = [
        ("Season: Summer", df['Season'] == 'Summer'),
        ("Season: Autumn", df['Season'] == 'Autumn'),
        ("Season: Winter", df['Season'] == 'Winter'),
        ("Season: Spring", df['Season'] == 'Spring'),
        ("Time: Day", df['Is_Day']),
        ("Time: Night", ~df['Is_Day']),
        ("Wind: Low (<6 m/s)", df['wind_speed_100m'] < 6),
        ("Wind: Medium (6-10 m/s)", (df['wind_speed_100m'] >= 6) & (df['wind_speed_100m'] < 10)),
        ("Wind: High (>10 m/s)", df['wind_speed_100m'] >= 10),
    ]

    for name, mask in tqdm(subgroups, desc="Subgroups", leave=False):
        sub_df = df[mask].copy()
        res = process_subgroup(sub_df, region_name, name, global_model_y, global_model_t)
        if res:
            all_results.append(res)

# MAIN EXECUTION
print("=" * 60)
print("PHASE 3: DOUBLE MACHINE LEARNING CAUSAL INFERENCE")
print("=" * 60)

for region, filename in FILES.items():
    file_path = os.path.join(DATA_DIR, filename)
    if os.path.exists(file_path):
        print(f"\nLoading {region}...")
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.set_index('timestamp')

        # Filter curtailment
        df_clean = df[~((df['wind_speed_100m'] > 5) & (df['power_mw'] <= 0.1))].copy()
        run_dml_analysis(region, df_clean)
    else:
        print(f"File not found: {file_path}")

# EXPORT & VISUALIZE
print("\n" + "=" * 60)
print("EXPORTING RESULTS")
print("=" * 60)

results_df = pd.DataFrame(all_results)

if not results_df.empty:
    out_path = os.path.join(TABLE_DIR, "DML_Causal_Effects_Summary.csv")
    results_df.to_csv(out_path, index=False)
    print(f"Results saved to: {out_path}")

    # Minimal Forest Plot Function
    def plot_forest_minimal(df_res, title, filename):
        plt.figure(figsize=(10, 5))
        df_res = df_res.copy()

        def clean_label(s):
            if ":" in s:
                return s.split(":", 1)[1].strip()
            return s

        df_res['Label'] = df_res['Subgroup'].apply(clean_label)
        df_res = df_res.sort_values(by=['Subgroup', 'Region'])

        for region in sorted(df_res['Region'].unique()):
            subset = df_res[df_res['Region'] == region]
            color = '#FF8C00' if region == "North Island" else '#1E90FF'

            y_pos = np.arange(len(subset))
            offset = 0.15 if region == "North Island" else -0.15

            plt.errorbar(
                subset[EFFECT_COL],
                y_pos + offset,
                xerr=(subset[EFFECT_COL] - subset['CI_Lower']),
                fmt='o', color=color, label=region, capsize=4,
                alpha=0.9, markersize=8, linewidth=2
            )

            if region == "North Island":
                plt.yticks(y_pos, subset['Label'])

        plt.axvline(x=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
        plt.title(title, fontsize=12, fontweight='bold')
        plt.xlabel("Causal Effect (MW per 0.1 kg/m³)", fontsize=10)
        plt.grid(True, alpha=0.2, axis='x')
        plt.legend(loc='lower right', fontsize=10)
        plt.tight_layout()

        save_path = os.path.join(PLOT_DIR, filename)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved plot: {filename}")

    # Generate Plots
    seasonal = results_df[results_df['Subgroup'].str.contains("Season", na=False)]
    if not seasonal.empty:
        plot_forest_minimal(seasonal, "Seasonal Heterogeneity", "Forest_Plot_Seasonal.png")

    wind = results_df[results_df['Subgroup'].str.contains("Wind", na=False)]
    if not wind.empty:
        plot_forest_minimal(wind, "Wind Regime Heterogeneity", "Forest_Plot_WindRegime.png")

    diurnal = results_df[results_df['Subgroup'].str.contains("Time", na=False)]
    if not diurnal.empty:
        plot_forest_minimal(diurnal, "Diurnal Heterogeneity", "Forest_Plot_Diurnal.png")

    print(f"\nTotal results generated: {len(results_df)} rows")
else:
    print("No results generated")

print("\n" + "=" * 60)
print("PHASE 3 COMPLETE")
print("=" * 60)

## **Phase-04**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.base import clone
from scipy import stats
from statsmodels.stats.stattools import durbin_watson


ISLAND_COLORS = {
    "North Island": "#FF8C00",
    "South Island": "#1E90FF"
}

# Causal Model Definition
Y_COL = 'power_mw'
T_COL = 'air_density_kgm3'
# Excluded temp/pressure to avoid multicollinearity
X_COLS = [
    'wind_speed_100m', 'wind_cubed', 'turbulence_proxy', 'ramp_rate', 'wind_lag_1',
    'wind_sin', 'wind_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos'
]

robustness_results = []

# MANUAL DML ENGINE (Consistent with Phase 3)
def run_dml_manual(df, model_y, model_t, n_folds=5, treatment_col=T_COL):
    """
    Manual DML implementation using K-fold cross-fitting.
    Matches Phase 3 methodology exactly.
    """
    # Block K-Fold (Respects Time Order, No Shuffling)
    kf = KFold(n_splits=n_folds, shuffle=False)

    # Prepare data matrices
    X_data = df[X_COLS].values
    y_data = df[Y_COL].values
    t_data = df[treatment_col].values

    residuals_y = np.zeros(len(df))
    residuals_t = np.zeros(len(df))

    # Cross-fitting loop
    for train_idx, test_idx in kf.split(X_data):
        X_train, X_test = X_data[train_idx], X_data[test_idx]
        y_train, y_test = y_data[train_idx], y_data[test_idx]
        t_train, t_test = t_data[train_idx], t_data[test_idx]

        # Fit Outcome Model
        m_y = clone(model_y)
        m_y.fit(X_train, y_train)
        residuals_y[test_idx] = y_test - m_y.predict(X_test)

        # Fit Treatment Model
        m_t = clone(model_t)
        m_t.fit(X_train, t_train)
        residuals_t[test_idx] = t_test - m_t.predict(X_test)

    # Final Regression (Residual on Residual)
    coef, intercept, r_value, p_value, se = stats.linregress(residuals_t, residuals_y)

    return {
        'Effect': coef,
        'SE': se,
        'P_Value': p_value,
        'Residuals_Y': residuals_y, # Return for diagnostics
        'Residuals_T': residuals_t
    }

# ROBUSTNESS SUITE
def plot_physics_validation(df, region, color):
    """4.1 Physics Validation: Visualizing the Density Effect."""
    print(f"Generating Physics Validation Plot...")
    plt.figure(figsize=(10, 6))

    # Filter for the "Ramp" section where density matters most (5-12 m/s)
    subset = df[(df['wind_speed_100m'] >= 5) & (df['wind_speed_100m'] <= 12)].copy()

    # Bin Density into High/Low
    high_rho = subset['air_density_kgm3'].quantile(0.90)
    low_rho = subset['air_density_kgm3'].quantile(0.10)

    subset['Density_Bin'] = np.nan
    subset.loc[subset['air_density_kgm3'] >= high_rho, 'Density_Bin'] = 'High Density (>90%)'
    subset.loc[subset['air_density_kgm3'] <= low_rho, 'Density_Bin'] = 'Low Density (<10%)'

    plot_data = subset.dropna(subset=['Density_Bin'])

    # Plot
    sns.lineplot(data=plot_data, x='wind_speed_100m', y='power_mw', hue='Density_Bin',
                 style='Density_Bin', markers=True, dashes=False, palette='dark')

    plt.title(f"Physics Validation: Power Curve by Air Density ({region})")
    plt.xlabel("Wind Speed (m/s)")
    plt.ylabel("Power Output (MW)")
    plt.grid(True, alpha=0.3)
    plt.legend(title="Air Density Regime")

    save_path = os.path.join(PLOT_DIR, f"Physics_Validation_{region.replace(' ', '_')}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def run_robustness_checks(region, df):
    print(f"Running Robustness Suite for {region}...")

    # Base Learner (XGBoost - Consistent with Phase 3)
    xgb_learner = XGBRegressor(n_estimators=100, max_depth=5, n_jobs=-1, random_state=42)

    # Placebo Test
    print("Running Placebo Test (Random Noise)...")
    # Create random noise variable
    df['placebo_noise'] = np.random.normal(0, 1, size=len(df))

    # Run DML treating Placebo as the Treatment
    res_placebo = run_dml_manual(df, xgb_learner, xgb_learner, treatment_col='placebo_noise')

    robustness_results.append({
        'Region': region,
        'Test_Type': 'Placebo',
        'Specification': 'Random Noise Treatment',
        'Effect': res_placebo['Effect'],
        'SE': res_placebo['SE'],
        'P_Value': res_placebo['P_Value']
    })

    # Temporal Stability (Date Split)
    print("Running Temporal Stability Test (Pre/Post 2015)")
    split_date = '2015-01-01'
    df_early = df[df.index < split_date]
    df_late = df[df.index >= split_date]

    # Early Era
    if len(df_early) > 5000:
        res_early = run_dml_manual(df_early, xgb_learner, xgb_learner)
        robustness_results.append({
            'Region': region,
            'Test_Type': 'Temporal Stability',
            'Specification': 'Early Era (2005-2014)',
            'Effect': res_early['Effect'],
            'SE': res_early['SE'],
            'P_Value': res_early['P_Value']
        })

    # Late Era
    if len(df_late) > 5000:
        res_late = run_dml_manual(df_late, xgb_learner, xgb_learner)
        robustness_results.append({
            'Region': region,
            'Test_Type': 'Temporal Stability',
            'Specification': 'Late Era (2015-2024)',
            'Effect': res_late['Effect'],
            'SE': res_late['SE'],
            'P_Value': res_late['P_Value']
        })

    # Learner Robustness
    print("Running Learner Robustness (RF & OLS)")

    # Random Forest
    rf_learner = RandomForestRegressor(n_estimators=50, max_depth=7, n_jobs=-1, random_state=42)
    res_rf = run_dml_manual(df, rf_learner, rf_learner)

    robustness_results.append({
        'Region': region,
        'Test_Type': 'Learner Robustness',
        'Specification': 'Random Forest',
        'Effect': res_rf['Effect'],
        'SE': res_rf['SE'],
        'P_Value': res_rf['P_Value']
    })

    # Linear Regression
    ols_learner = LinearRegression()
    res_ols = run_dml_manual(df, ols_learner, ols_learner)

    robustness_results.append({
        'Region': region,
        'Test_Type': 'Learner Robustness',
        'Specification': 'Linear Regression (OLS)',
        'Effect': res_ols['Effect'],
        'SE': res_ols['SE'],
        'P_Value': res_ols['P_Value']
    })

    # Residual Diagnostics (Durbin-Watson)
    print("Calculating Residual Autocorrelation (Durbin-Watson)...")
    # We use the residuals from the Main XGBoost
    res_main = run_dml_manual(df, xgb_learner, xgb_learner)

    # Durbin-Watson on the Final DML Residuals (Y_resid - theta * T_resid)
    dw_stat = durbin_watson(res_main['Residuals_Y'])

    robustness_results.append({
        'Region': region,
        'Test_Type': 'Diagnostics',
        'Specification': 'Durbin-Watson Statistic',
        'Effect': dw_stat,
        'SE': np.nan,
        'P_Value': np.nan
    })


# EXECUTION LOOP
for region, filename in FILES.items():
    file_path = os.path.join(DATA_DIR, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.set_index('timestamp')

        # Filter curtailment
        df_clean = df[~((df['wind_speed_100m'] > 5) & (df['power_mw'] <= 0.1))].copy()
        df_clean = df_clean.dropna(subset=[Y_COL, T_COL] + X_COLS)

        # Physics Plot
        plot_physics_validation(df_clean, region, ISLAND_COLORS[region])

        # Run Suite
        run_robustness_checks(region, df_clean)


# EXPORT RESULTS
results_df = pd.DataFrame(robustness_results)
out_path = os.path.join(TABLE_DIR, "Robustness_Validation_Summary.csv")
results_df.to_csv(out_path, index=False)

print("\nPhase 4 Complete.")
print(f"Robustness Table saved to: {out_path}")
print(f"Validation Plots saved to: {PLOT_DIR}")

# Print Summary
print("\nSummary of Robustness Checks:")
print(results_df[['Region', 'Test_Type', 'Specification', 'Effect']].to_string(index=False))

### **SOUTH ISLAND ERA-STRATIFIED VALIDATION & DIAGNOSTICS**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from scipy.stats import spearmanr, kruskal
from sklearn.linear_model import LinearRegression

Y_COL = 'power_mw'
T_COL = 'air_density_kgm3'
X_COLS = [
    'wind_speed_100m', 'wind_cubed', 'turbulence_proxy', 'ramp_rate', 'wind_lag_1',
    'wind_sin', 'wind_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos'
]


# HELPER: Load & Split Data
def load_and_split_south_island():
    """Load South Island data and split by 2015 date."""
    file_path = os.path.join(DATA_DIR, "ENHANCED_DATASET_SOUTH_ISLAND.csv")
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('timestamp')

    # Filter curtailment
    df_clean = df[~((df['wind_speed_100m'] > 5) & (df['power_mw'] <= 0.1))].copy()
    df_clean = df_clean.dropna(subset=[Y_COL, T_COL] + X_COLS)

    # Split by date
    split_date = '2015-01-01'
    df_early = df_clean[df_clean.index < split_date]
    df_late = df_clean[df_clean.index >= split_date]

    print(f"Early data: {df_early.index.min()} to {df_early.index.max()} (n={len(df_early):,})")
    print(f"Late data:  {df_late.index.min()} to {df_late.index.max()} (n={len(df_late):,})")

    return df_clean, df_early, df_late



# ERA DIAGNOSTICS TABLE
def diagnostic_table_by_era(df_early, df_late, df_all):
    """Compare key statistics across eras."""
    print("\n" + "=" * 80)
    print("SOUTH ISLAND ERA DIAGNOSTICS")
    print("=" * 80)

    diagnostics = []

    for era_name, df_era in [("Early (2005-2014)", df_early),
                              ("Late (2015-2024)", df_late),
                              ("All (2005-2024)", df_all)]:

        n = len(df_era)
        power_mean = df_era[Y_COL].mean()
        power_std = df_era[Y_COL].std()
        power_min = df_era[Y_COL].min()
        power_max = df_era[Y_COL].max()

        density_mean = df_era[T_COL].mean()
        density_std = df_era[T_COL].std()
        wind_mean = df_era['wind_speed_100m'].mean()

        # Seasonal composition
        season_map = {
            12: 'Summer', 1: 'Summer', 2: 'Summer',
            3: 'Autumn', 4: 'Autumn', 5: 'Autumn',
            6: 'Winter', 7: 'Winter', 8: 'Winter',
            9: 'Spring', 10: 'Spring', 11: 'Spring'
        }
        df_era_copy = df_era.copy()
        df_era_copy['Season'] = df_era_copy.index.month.map(season_map)
        summer_pct = (df_era_copy['Season'] == 'Summer').sum() / n * 100

        diagnostics.append({
            'Era': era_name,
            'N': f"{n:,}",
            'Power_Mean_MW': f"{power_mean:.2f}",
            'Power_Range_MW': f"{power_min:.1f}–{power_max:.1f}",
            'Density_Mean': f"{density_mean:.4f}",
            'Density_Std': f"{density_std:.4f}",
            'Wind_Mean_ms': f"{wind_mean:.2f}",
            'Summer_pct': f"{summer_pct:.1f}%"
        })

    diag_df = pd.DataFrame(diagnostics)
    print(diag_df.to_string(index=False))

    # Save
    diag_path = os.path.join(TABLE_DIR, "South_Island_Era_Diagnostics.csv")
    diag_df.to_csv(diag_path, index=False)
    print(f"\nSaved to: {diag_path}")

    return diag_df

# CONFOUNDER BALANCE BY ERA
def confounder_balance_by_era(df_early, df_late):
    """Check if confounders have different distributions by era."""
    print("\n" + "=" * 80)
    print("CONFOUNDER BALANCE BY ERA (Kruskal-Wallis Test)")
    print("=" * 80)

    balance_results = []

    for col in X_COLS:
        # Kruskal-Wallis test
        stat, p_val = kruskal(df_early[col].dropna(), df_late[col].dropna())

        mean_early = df_early[col].mean()
        mean_late = df_late[col].mean()
        pct_diff = ((mean_late - mean_early) / abs(mean_early)) * 100 if mean_early != 0 else 0

        balance_results.append({
            'Confounder': col,
            'Mean_Early': f"{mean_early:.4f}",
            'Mean_Late': f"{mean_late:.4f}",
            'Pct_Diff': f"{pct_diff:+.2f}%",
            'P_Value': f"{p_val:.4f}",
            'Sig': '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'NS'
        })

    balance_df = pd.DataFrame(balance_results)
    print(balance_df.to_string(index=False))

    # Save
    balance_path = os.path.join(TABLE_DIR, "South_Island_Confounder_Balance.csv")
    balance_df.to_csv(balance_path, index=False)
    print(f"\nSaved to: {balance_path}")

# CORRELATION STRUCTURE BY ERA
def correlation_by_era(df_early, df_late):
    """Compare correlations between Density and Power by era."""
    print("\n" + "=" * 80)
    print("PARTIAL CORRELATION: Density → Power by Era")
    print("=" * 80)

    corr_results = []

    for era_name, df_era in [("Early (2005-2014)", df_early),
                              ("Late (2015-2024)", df_late)]:

        # Raw correlation
        raw_corr, raw_p = spearmanr(df_era[T_COL], df_era[Y_COL])

        # Control for wind speed (primary confounder)
        lr_y = LinearRegression().fit(df_era[['wind_speed_100m']], df_era[Y_COL])
        lr_t = LinearRegression().fit(df_era[['wind_speed_100m']], df_era[T_COL])

        y_resid = df_era[Y_COL] - lr_y.predict(df_era[['wind_speed_100m']])
        t_resid = df_era[T_COL] - lr_t.predict(df_era[['wind_speed_100m']])

        partial_corr, partial_p = spearmanr(t_resid, y_resid)

        corr_results.append({
            'Era': era_name,
            'Raw_Corr': f"{raw_corr:.4f}",
            'Raw_P': f"{raw_p:.4f}",
            'Partial_Corr': f"{partial_corr:.4f}",
            'Partial_P': f"{partial_p:.4f}",
            'Sign_Flip': 'YES' if (raw_corr * partial_corr < 0) else 'NO'
        })

    corr_df = pd.DataFrame(corr_results)
    print(corr_df.to_string(index=False))

    # Save
    corr_path = os.path.join(TABLE_DIR, "South_Island_Correlation_by_Era.csv")
    corr_df.to_csv(corr_path, index=False)
    print(f"\n✓ Saved to: {corr_path}")

# SEASONAL COMPOSITION ANALYSIS
def seasonal_analysis(df_early, df_late, df_all):
    """Check if seasonal composition differs between eras."""
    print("\n" + "=" * 80)
    print("SEASONAL COMPOSITION BY ERA")
    print("=" * 80)

    season_map = {
        12: 'Summer', 1: 'Summer', 2: 'Summer',
        3: 'Autumn', 4: 'Autumn', 5: 'Autumn',
        6: 'Winter', 7: 'Winter', 8: 'Winter',
        9: 'Spring', 10: 'Spring', 11: 'Spring'
    }

    seasonal_results = []

    for era_name, df_era in [("Early (2005-2014)", df_early),
                              ("Late (2015-2024)", df_late),
                              ("All (2005-2024)", df_all)]:

        df_era_copy = df_era.copy()
        df_era_copy['Season'] = df_era_copy.index.month.map(season_map)

        for season in ['Summer', 'Autumn', 'Winter', 'Spring']:
            season_data = df_era_copy[df_era_copy['Season'] == season]

            if len(season_data) > 0:
                pct = len(season_data) / len(df_era) * 100
                seasonal_results.append({
                    'Era': era_name,
                    'Season': season,
                    'Pct_of_Era': f"{pct:.1f}%",
                    'N': f"{len(season_data):,}"
                })

    seasonal_df = pd.DataFrame(seasonal_results)
    print(seasonal_df.to_string(index=False))

    # Chi-square test
    early_seasonal = df_early.copy()
    early_seasonal['Season'] = early_seasonal.index.month.map(season_map)
    early_counts = early_seasonal['Season'].value_counts().sort_index().values  # FIX: Add .values

    late_seasonal = df_late.copy()
    late_seasonal['Season'] = late_seasonal.index.month.map(season_map)
    late_counts = late_seasonal['Season'].value_counts().sort_index().values  # FIX: Add .values

    chi2, p = stats.chisquare(
        f_obs=late_counts,
        f_exp=(early_counts / early_counts.sum() * late_counts.sum())
    )

    print(f"\nChi-square test for seasonal distribution difference:")
    print(f"  χ² = {chi2:.3f}, p-value = {p:.4f}")

    # Save
    seasonal_path = os.path.join(TABLE_DIR, "South_Island_Seasonal_by_Era.csv")
    seasonal_df.to_csv(seasonal_path, index=False)
    print(f"Saved to: {seasonal_path}")

# VISUALIZATION: Density vs Power by Era
def plot_density_power_by_era(df_early, df_late):
    """Scatterplot of Density vs Power, colored by era."""
    print("\n" + "=" * 80)
    print("GENERATING VISUALIZATION: Density vs Power by Era")
    print("=" * 80)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Early Era
    axes[0].scatter(df_early[T_COL], df_early[Y_COL], alpha=0.3, s=10, color='blue')
    z_early = np.polyfit(df_early[T_COL].dropna(), df_early[Y_COL].dropna(), 1)
    p_early = np.poly1d(z_early)
    x_range = np.linspace(df_early[T_COL].min(), df_early[T_COL].max(), 100)
    axes[0].plot(x_range, p_early(x_range), 'b-', linewidth=3, label=f'Slope: {z_early[0]:.1f} MW/(kg/m³)')
    axes[0].set_xlabel('Air Density (kg/m³)', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Power (MW)', fontsize=12, fontweight='bold')
    axes[0].set_title('Early Era (2005-2014)', fontsize=13, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)

    # Late Era
    axes[1].scatter(df_late[T_COL], df_late[Y_COL], alpha=0.3, s=10, color='red')
    z_late = np.polyfit(df_late[T_COL].dropna(), df_late[Y_COL].dropna(), 1)
    p_late = np.poly1d(z_late)
    x_range = np.linspace(df_late[T_COL].min(), df_late[T_COL].max(), 100)
    axes[1].plot(x_range, p_late(x_range), 'r-', linewidth=3, label=f'Slope: {z_late[0]:.1f} MW/(kg/m³)')
    axes[1].set_xlabel('Air Density (kg/m³)', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Power (MW)', fontsize=12, fontweight='bold')
    axes[1].set_title('Late Era (2015-2024)', fontsize=13, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)

    # Overlay
    axes[2].scatter(df_early[T_COL], df_early[Y_COL], alpha=0.2, s=10, color='blue', label='Early')
    axes[2].scatter(df_late[T_COL], df_late[Y_COL], alpha=0.2, s=10, color='red', label='Late')
    axes[2].plot(x_range, p_early(x_range), 'b-', linewidth=3, label=f'Early: {z_early[0]:.1f}')
    axes[2].plot(x_range, p_late(x_range), 'r-', linewidth=3, label=f'Late: {z_late[0]:.1f}')
    axes[2].set_xlabel('Air Density (kg/m³)', fontsize=12, fontweight='bold')
    axes[2].set_ylabel('Power (MW)', fontsize=12, fontweight='bold')
    axes[2].set_title('Overlay Comparison', fontsize=13, fontweight='bold')
    axes[2].legend(fontsize=10)
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plot_path = os.path.join(PLOT_DIR, "South_Island_Density_Power_by_Era.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved to: {plot_path}")

# MAIN EXECUTION
if __name__ == "__main__":
    print("=" * 80)
    print("SOUTH ISLAND ERA-STRATIFIED VALIDATION")
    print("=" * 80)

    # Load data
    df_all, df_early, df_late = load_and_split_south_island()

    # Run diagnostics
    diagnostic_table_by_era(df_early, df_late, df_all)
    confounder_balance_by_era(df_early, df_late)
    correlation_by_era(df_early, df_late)
    seasonal_analysis(df_early, df_late, df_all)
    plot_density_power_by_era(df_early, df_late)

    print("\n" + "=" * 80)
    print("SOUTH ISLAND ERA-STRATIFIED VALIDATION COMPLETE")
    print("=" * 80)


## **Phase-05**

In [ ]:
# SHAP INTERPRETABILITY

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shap
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")

# Global styling
plt.rcParams.update({
    "figure.dpi": 180,
    "savefig.dpi": 300,
    "font.size": 12,
    "axes.titlesize": 14,
    "axes.titlepad": 18,
    "axes.labelsize": 12,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "figure.facecolor": "white",
    "axes.facecolor": "white",
})

Y_COL = "power_mw"
T_COL = "air_density_kgm3"
X_COLS = [
    "wind_speed_100m", "wind_cubed", "turbulence_proxy", "ramp_rate", "wind_lag_1",
    "wind_sin", "wind_cos", "hour_sin", "hour_cos", "month_sin", "month_cos"
]

SEASON_MAP = {
    12: "Summer", 1: "Summer", 2: "Summer",
    3: "Autumn", 4: "Autumn", 5: "Autumn",
    6: "Winter", 7: "Winter", 8: "Winter",
    9: "Spring", 10: "Spring", 11: "Spring"
}


# Helpers
def tune_hyperparameters(X, y):
    param_grid = {
        "n_estimators": [100, 200],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.05, 0.1],
        "subsample": [0.8, 0.9],
        "colsample_bytree": [0.8, 0.9],
    }
    model = XGBRegressor(n_jobs=-1, random_state=42)
    tscv = TimeSeriesSplit(n_splits=3)

    search = RandomizedSearchCV(
        model,
        param_distributions=param_grid,
        n_iter=5,
        cv=tscv,
        scoring="neg_mean_squared_error",
        random_state=42,
        n_jobs=-1,
        verbose=0,
    )
    search.fit(X, y)
    return search.best_estimator_

def save_fig(path, fig=None, pad=0.60, rect=(0.03, 0.03, 0.97, 0.92)):
    """
    Unified save method with extra padding to avoid clipping (titles/colorbars).
    rect=(left, bottom, right, top) reserved area for tight_layout.
    """
    if fig is None:
        fig = plt.gcf()
    try:
        fig.tight_layout(rect=list(rect))
    except Exception:
        pass
    fig.savefig(path, bbox_inches="tight", pad_inches=pad, facecolor="white")
    plt.close(fig)

def stratified_sample(df, y_col, max_n=10000, n_tail=500, seed=42):
    """Tail-aware sampling: top + bottom + random remainder."""
    if len(df) <= max_n:
        return df.copy()

    top_power = df.nlargest(n_tail, y_col)
    low_power = df.nsmallest(n_tail, y_col)
    random_n = max_n - 2 * n_tail
    random_sample = df.sample(n=random_n, random_state=seed)

    out = pd.concat([top_power, low_power, random_sample]).drop_duplicates()
    if len(out) > max_n:
        out = out.sample(n=max_n, random_state=seed)
    return out

# Phase-05 runner
def run_shap_analysis(region, filename):
    print(f"\n{'='*70}\nSHAP Analysis: {region}\n{'='*70}")

    path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(path):
        print(f"File not found: {path}")
        return

    # Load & prep
    df = pd.read_csv(path)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.set_index("timestamp").sort_index()
    df = df.dropna(subset=[Y_COL, T_COL] + X_COLS)

    # Season (before sampling)
    df["Season"] = df.index.month.map(SEASON_MAP)

    # Stratified sampling
    shap_df = stratified_sample(df, Y_COL, max_n=10000, n_tail=500, seed=42).reset_index(drop=True)

    X = shap_df[X_COLS]
    y = shap_df[Y_COL]

    # Train
    print("Training outcome model (XGBoost)")
    model_y = tune_hyperparameters(X, y)

    # ---- SHAP
    print("Computing SHAP values")
    explainer = shap.TreeExplainer(model_y)
    shap_values = explainer(X)


    # Summary plot (robust)
    print("Saving summary plot")
    plt.figure(figsize=(14, 10))
    shap.summary_plot(
        shap_values, X,
        show=False,
        plot_type="dot",
        cmap=plt.get_cmap("autumn" if region == "North Island" else "winter"),
    )
    plt.title(f"SHAP Summary: Drivers of Power Generation\n{region}",
              fontsize=16, fontweight="bold", pad=28)
    save_fig(os.path.join(PLOT_DIR, f"SHAP_Summary_Outcome_{region.replace(' ', '_')}.png"),
             pad=0.65, rect=(0.03, 0.03, 0.97, 0.90))

    # Dependence plot
    print("Saving dependence plot")
    plt.figure(figsize=(12, 9))
    shap.plots.scatter(
        shap_values[:, "wind_speed_100m"],
        color=shap_values[:, "turbulence_proxy"],
        show=False
    )
    plt.title(f"SHAP Dependence: Wind Speed Impact on Power\n{region}",
              fontsize=16, fontweight="bold", pad=26)
    save_fig(os.path.join(PLOT_DIR, f"SHAP_Dependence_Wind_{region.replace(' ', '_')}.png"),
             pad=0.65, rect=(0.03, 0.03, 0.97, 0.90))

    # Seasonal comparison
    print("Saving seasonal comparison (Summer vs Winter)")
    summer_idx = np.where((shap_df["Season"] == "Summer").values)[0]
    winter_idx = np.where((shap_df["Season"] == "Winter").values)[0]

    if (len(summer_idx) > 50) and (len(winter_idx) > 50):
        fig, axes = plt.subplots(
            1, 2,
            figsize=(28, 9),
            gridspec_kw={"wspace": 0.95}
        )

        # Summer
        plt.sca(axes[0])
        shap.plots.bar(shap_values[summer_idx], max_display=10, show=False)
        axes[0].set_title(
            f"Feature Importance\nSummer ({region})\nN={len(summer_idx)}",
            fontsize=13, fontweight="bold", pad=16
        )

        # Winter
        plt.sca(axes[1])
        shap.plots.bar(shap_values[winter_idx], max_display=10, show=False)
        axes[1].set_title(
            f"Feature Importance\nWinter ({region})\nN={len(winter_idx)}",
            fontsize=13, fontweight="bold", pad=16
        )

        fig.suptitle(
            f"Seasonal Feature Importance Comparison\n{region}",
            fontsize=18, fontweight="bold", y=0.98
        )

        # Explicit spacing control:
        fig.subplots_adjust(
            top=0.82,
            bottom=0.10,
            left=0.08,
            right=0.98,
            wspace=0.95
        )

        out_path = os.path.join(PLOT_DIR, f"SHAP_Seasonal_Compare_{region.replace(' ', '_')}.png")
        fig.savefig(out_path, dpi=300, bbox_inches="tight", pad_inches=0.70, facecolor="white")
        plt.close(fig)
    else:
        print(f"Skipping seasonal plot (summer={len(summer_idx)}, winter={len(winter_idx)})")


    # Waterfall: High power event
    print("Saving waterfall (high power event)")
    high_idx = int(np.argmax(y.values))
    plt.figure(figsize=(14, 10))
    shap.plots.waterfall(shap_values[high_idx], max_display=12, show=False)
    plt.title(f"Local Explanation: High Power Event\n{region} | Power={y.iloc[high_idx]:.1f} MW",
              fontsize=16, fontweight="bold", pad=28)
    save_fig(os.path.join(PLOT_DIR, f"SHAP_Waterfall_High_{region.replace(' ', '_')}.png"),
             pad=0.70, rect=(0.03, 0.03, 0.97, 0.90))

    # Waterfall: Low power event
    print("Saving waterfall (low power event)")
    low_idx = int(np.argmin(y.values))
    plt.figure(figsize=(14, 10))
    shap.plots.waterfall(shap_values[low_idx], max_display=12, show=False)
    plt.title(f"Local Explanation: Low Power Event\n{region} | Power={y.iloc[low_idx]:.1f} MW",
              fontsize=16, fontweight="bold", pad=28)
    save_fig(os.path.join(PLOT_DIR, f"SHAP_Waterfall_Low_{region.replace(' ', '_')}.png"),
             pad=0.70, rect=(0.03, 0.03, 0.97, 0.90))

    print(f"Done: {region} (5 plots saved)")

# Execute
print("="*80)
print("PHASE 05: SHAP ANALYSIS")
print("="*80)

for region_name, fname in FILES.items():
    run_shap_analysis(region_name, fname)

print("\nAll Phase-05 plots saved to:")
print(PLOT_DIR)

## **Phase-06**

In [ ]:
# PHASE 06: Sensitivity + Time-series Robust Inference + Overlap Diagnostics

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.base import clone
from sklearn.metrics import r2_score
from scipy import stats

ISLAND_COLORS = {
    "North Island": "#FF8C00",
    "South Island": "#1E90FF"
}

Y_COL = "power_mw"
T_COL = "air_density_kgm3"
X_COLS = [
    "wind_speed_100m", "wind_cubed", "turbulence_proxy", "ramp_rate", "wind_lag_1",
    "wind_sin", "wind_cos", "hour_sin", "hour_cos", "month_sin", "month_cos"
]

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# Operational-zero candidate threshold
POWER_ZERO_THR = 0.1
HIGH_WIND_THR = 8.0

# Time-series bootstrap controls
BOOT_B = 300
BLOCK_LEN_HOURS = 168


# Utilities
def load_region_df(region, filename):
    path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.set_index("timestamp").sort_index()

    cols = [Y_COL, T_COL] + X_COLS
    df = df.dropna(subset=cols)

    # Keep physically plausible non-negative power
    df = df[df[Y_COL] >= 0].copy()

    return df

def tune_xgb_regressor(X, y, n_iter=8):
    param_grid = {
        "n_estimators": [150, 250, 350],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.03, 0.05, 0.1],
        "subsample": [0.8, 0.9, 1.0],
        "colsample_bytree": [0.8, 0.9, 1.0],
        "min_child_weight": [1, 3, 5],
        "reg_lambda": [1.0, 2.0, 5.0]
    }

    base = XGBRegressor(
        n_jobs=-1,
        random_state=RANDOM_SEED,
        tree_method="hist"
    )

    tscv = TimeSeriesSplit(n_splits=3)
    search = RandomizedSearchCV(
        base,
        param_distributions=param_grid,
        n_iter=n_iter,
        cv=tscv,
        scoring="neg_mean_squared_error",
        random_state=RANDOM_SEED,
        n_jobs=-1,
        verbose=0
    )
    search.fit(X, y)
    return search.best_estimator_

def detect_operational_zeros(df):
    return (df[Y_COL] <= POWER_ZERO_THR) & (df["wind_speed_100m"] >= HIGH_WIND_THR)

def apply_scenario(df, scenario_name):
    d = df.copy()

    if scenario_name == "A_base_filtered":
        d = d[(d["wind_speed_100m"] > 5.0) & (d[Y_COL] > POWER_ZERO_THR)].copy()

    elif scenario_name == "B_keep_zeros_wind_gt5":
        d = d[(d["wind_speed_100m"] > 5.0)].copy()

    elif scenario_name == "C_all_data_no_wind_filter":
        d = d.copy()

    elif scenario_name == "D_exclude_operational_zeros":
        # drop only high-wind near-zero power candidates
        d = d[~detect_operational_zeros(d)].copy()

    else:
        raise ValueError(f"Unknown scenario: {scenario_name}")

    d = d.dropna(subset=[Y_COL, T_COL] + X_COLS)
    return d

def crossfit_residuals_timeseries(df, model_y, model_t, n_splits=5):
    X = df[X_COLS].values
    y = df[Y_COL].values
    t = df[T_COL].values

    tscv = TimeSeriesSplit(n_splits=n_splits)

    ry = np.zeros(len(df))
    rt = np.zeros(len(df))

    for train_idx, test_idx in tscv.split(X):
        my = clone(model_y)
        mt = clone(model_t)

        my.fit(X[train_idx], y[train_idx])
        mt.fit(X[train_idx], t[train_idx])

        ry[test_idx] = y[test_idx] - my.predict(X[test_idx])
        rt[test_idx] = t[test_idx] - mt.predict(X[test_idx])

    return ry, rt

def residual_on_residual_effect(rt, ry):
    # OLS slope + naive SE from scipy linregress
    slope, intercept, rvalue, pvalue, stderr = stats.linregress(rt, ry)
    ci_low = slope - 1.96 * stderr
    ci_high = slope + 1.96 * stderr
    return slope, stderr, pvalue, ci_low, ci_high

def moving_block_bootstrap_slopes(rt, ry, block_len, B, rng):
    n = len(rt)
    if n < 5 * block_len:
        # fallback to smaller blocks if series is short
        block_len = max(24, int(n / 10))

    n_blocks = int(np.ceil(n / block_len))
    max_start = n - block_len
    if max_start < 1:
        max_start = 1

    slopes = np.empty(B, dtype=float)

    for b in range(B):
        starts = rng.integers(0, max_start, size=n_blocks)
        idx = np.concatenate([np.arange(s, s + block_len) for s in starts])[:n]
        slope_b, _, _, _, _ = residual_on_residual_effect(rt[idx], ry[idx])
        slopes[b] = slope_b

    se_b = slopes.std(ddof=1)
    ci_low, ci_high = np.percentile(slopes, [2.5, 97.5])

    # two-sided bootstrap p-value
    p_boot = 2.0 * min(np.mean(slopes <= 0.0), np.mean(slopes >= 0.0))
    p_boot = float(min(1.0, p_boot))

    return se_b, ci_low, ci_high, p_boot, slopes

def fit_r2_diagnostics(df, model_y, model_t):
    X = df[X_COLS]
    y = df[Y_COL]
    t = df[T_COL]

    my = clone(model_y).fit(X, y)
    mt = clone(model_t).fit(X, t)

    r2y = r2_score(y, my.predict(X))
    r2t = r2_score(t, mt.predict(X))

    return float(r2y), float(r2t)


# Scenarios
SCENARIOS = [
    "A_base_filtered",
    "B_keep_zeros_wind_gt5",
    "C_all_data_no_wind_filter",
    "D_exclude_operational_zeros"
]


# Run Phase-06
results_rows = []
overlap_rows = []
opzero_examples = []

print("Phase-06 started")

for region, fname in FILES.items():
    print(f"Region: {region}")

    df0 = load_region_df(region, fname)

    # Tune global nuisance learners ONCE per region (use a time-sorted sample)
    df_tune = df0.copy()
    if len(df_tune) > 50000:
        df_tune = df_tune.sample(n=50000, random_state=RANDOM_SEED).sort_index()

    print("Tuning nuisance models")
    model_y_global = tune_xgb_regressor(df_tune[X_COLS], df_tune[Y_COL], n_iter=8)
    model_t_global = tune_xgb_regressor(df_tune[X_COLS], df_tune[T_COL], n_iter=8)

    # Operational-zero candidate summary on FULL data (before scenario filtering)
    oz_mask_full = detect_operational_zeros(df0)
    oz_rate_full = float(oz_mask_full.mean())
    oz_n_full = int(oz_mask_full.sum())

    # Save some examples (top wind speeds among operational-zero candidates)
    if oz_n_full > 0:
        df_oz = df0.loc[oz_mask_full, [Y_COL, T_COL] + X_COLS].copy()
        df_oz["region"] = region
        df_oz = df_oz.sort_values("wind_speed_100m", ascending=False).head(200)
        df_oz = df_oz.reset_index().rename(columns={"index": "timestamp"})
        opzero_examples.append(df_oz)

    for scen in SCENARIOS:
        df = apply_scenario(df0, scen)

        n = len(df)
        if n < 5000:
            # Keep consistent with earlier practice
            continue

        oz_mask = detect_operational_zeros(df)
        oz_rate = float(oz_mask.mean())
        oz_n = int(oz_mask.sum())

        # Cross-fit residuals
        ry, rt = crossfit_residuals_timeseries(df, model_y_global, model_t_global, n_splits=5)

        # Effect + naive uncertainty
        coef, se_naive, p_naive, ci_low_naive, ci_high_naive = residual_on_residual_effect(rt, ry)

        # Time-series robust uncertainty
        se_boot, ci_low_boot, ci_high_boot, p_boot, _ = moving_block_bootstrap_slopes(
            rt, ry,
            block_len=BLOCK_LEN_HOURS,
            B=BOOT_B,
            rng=rng
        )

        # Diagnostics
        r2y, r2t = fit_r2_diagnostics(df, model_y_global, model_t_global)

        # Overlap signals
        rt_std = float(np.std(rt, ddof=1))
        t_std = float(np.std(df[T_COL].values, ddof=1))
        rt_ratio = float(rt_std / t_std) if t_std > 0 else np.nan

        results_rows.append({
            "Region": region,
            "Scenario": scen,
            "N": n,
            "OperationalZeroCandidates_N": oz_n,
            "OperationalZeroCandidates_Rate": oz_rate,
            "Effect_MW_per_kgm3": float(coef),
            "Naive_SE": float(se_naive),
            "Naive_p": float(p_naive),
            "Naive_CI_Low": float(ci_low_naive),
            "Naive_CI_High": float(ci_high_naive),
            "BlockBootstrap_SE": float(se_boot),
            "BlockBootstrap_p": float(p_boot),
            "BlockBootstrap_CI_Low": float(ci_low_boot),
            "BlockBootstrap_CI_High": float(ci_high_boot),
            "R2_Outcome_inSample": float(r2y),
            "R2_Treatment_inSample": float(r2t),
            "Std_Treatment": float(t_std),
            "Std_TreatmentResidual": float(rt_std),
            "ResidualStd_to_TreatmentStd": float(rt_ratio),
            "BlockLenHours": int(BLOCK_LEN_HOURS),
            "BootstrapB": int(BOOT_B),
        })

        # Overlap diagnostics by wind-speed bins (where overlap can break)
        bins = [0, 3, 5, 7, 9, 11, 13, 50]
        labels = ["0-3", "3-5", "5-7", "7-9", "9-11", "11-13", "13+"]
        ws = df["wind_speed_100m"].values
        ws_bin = pd.cut(ws, bins=bins, labels=labels, include_lowest=True)

        tmp = pd.DataFrame({
            "ws_bin": ws_bin.astype(str),
            "t": df[T_COL].values,
            "rt": rt
        })

        grp = tmp.groupby("ws_bin", dropna=False).agg(
            N=("t", "size"),
            Std_T=("t", "std"),
            Std_RT=("rt", "std")
        ).reset_index()

        grp["Region"] = region
        grp["Scenario"] = scen
        overlap_rows.append(grp)

    print("Done")


# Save tables
res_df = pd.DataFrame(results_rows)
res_path = os.path.join(TABLE_DIR, "Phase06_DML_Sensitivity_Bootstrap.csv")
res_df.to_csv(res_path, index=False)

overlap_df = pd.concat(overlap_rows, ignore_index=True) if len(overlap_rows) else pd.DataFrame()
overlap_path = os.path.join(TABLE_DIR, "Phase06_Overlap_Diagnostics.csv")
overlap_df.to_csv(overlap_path, index=False)

if len(opzero_examples):
    opzero_df = pd.concat(opzero_examples, ignore_index=True)
    opzero_path = os.path.join(TABLE_DIR, "Phase06_OperationalZero_Candidates.csv")
    opzero_df.to_csv(opzero_path, index=False)

print("Saved:")
print(res_path)
print(overlap_path)
if len(opzero_examples):
    print(opzero_path)

# Operational-zero candidates scatter (one panel per region)
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

for i, (region, fname) in enumerate(FILES.items()):
    df0 = load_region_df(region, fname)
    c = ISLAND_COLORS[region]
    oz = detect_operational_zeros(df0)


    df_plot = df0[[Y_COL, "wind_speed_100m"]].copy()
    if len(df_plot) > 120000:
        df_plot = df_plot.sample(n=120000, random_state=RANDOM_SEED).sort_index()
        oz = oz.loc[df_plot.index]

    axes[i].scatter(df_plot["wind_speed_100m"], df_plot[Y_COL], s=6, alpha=0.12, color="gray", linewidths=0)
    if oz.any():
        axes[i].scatter(df_plot.loc[oz, "wind_speed_100m"], df_plot.loc[oz, Y_COL], s=8, alpha=0.6, color=c, linewidths=0)

    axes[i].set_title(f"Operational-zero candidates ({region})")
    axes[i].set_xlabel("wind_speed_100m (m/s)")
    if i == 0:
        axes[i].set_ylabel("power_mw (MW)")
    axes[i].grid(alpha=0.2)

fig.suptitle(f"Operational-zero candidates: power_mw <= {POWER_ZERO_THR} with wind_speed_100m >= {HIGH_WIND_THR}", y=1.02)
plt.tight_layout()
plot_path = os.path.join(PLOT_DIR, "Phase06_OperationalZeros_Scatter.png")
plt.savefig(plot_path, dpi=300, bbox_inches="tight", facecolor="white")
plt.close()

# Operational-zero rate by scenario (bar chart)
if not res_df.empty:
    plot_df = res_df.copy()
    plot_df["ScenarioLabel"] = plot_df["Scenario"].map({
        "A_base_filtered": "Base filtered",
        "B_keep_zeros_wind_gt5": "Keep zeros (wind>5)",
        "C_all_data_no_wind_filter": "All data",
        "D_exclude_operational_zeros": "Exclude op-zeros"
    })

    fig, ax = plt.subplots(figsize=(12, 5))

    # Arrange grouped bars
    scen_order = ["Base filtered", "Keep zeros (wind>5)", "All data", "Exclude op-zeros"]
    regions = ["North Island", "South Island"]
    x = np.arange(len(scen_order))
    w = 0.35

    for j, region in enumerate(regions):
        sub = plot_df[plot_df["Region"] == region].set_index("ScenarioLabel").reindex(scen_order)
        y = sub["OperationalZeroCandidates_Rate"].values
        ax.bar(x + (j - 0.5) * w, y, width=w, color=ISLAND_COLORS[region], alpha=0.85, label=region)

    ax.set_title("Operational-zero candidate rate by scenario")
    ax.set_xlabel("Scenario")
    ax.set_ylabel("Rate (fraction of hours)")
    ax.set_xticks(x)
    ax.set_xticklabels(scen_order, rotation=0)
    ax.grid(axis="y", alpha=0.2)
    ax.legend()

    plt.tight_layout()
    plot_path = os.path.join(PLOT_DIR, "Phase06_OperationalZero_Rate.png")
    plt.savefig(plot_path, dpi=300, bbox_inches="tight", facecolor="white")
    plt.close()

print("Plots saved:")
print(os.path.join(PLOT_DIR, "Phase06_OperationalZeros_Scatter.png"))
print(os.path.join(PLOT_DIR, "Phase06_OperationalZero_Rate.png"))

print("Phase-06 complete")